In [1]:
import pandas as pd

In [4]:
data = pd.read_csv("../output_data/seth/step3_description_data.csv")
data.head()

,source,domain,lineID,change_type,before/after,timestamp,timestamp_rule,communityID,ref,ruleID,text
0,earliest,reddit,0,unchanged,unchanged,2021-04-23 00:00:00,20210401,tslt,https://www.reddit.com/r/tslt/,tslt_0,"hey, we're easy going people. you can post you..."
1,earliest,reddit,0,unchanged,unchanged,2021-04-23 00:00:00,20210301,okanaganbiking,https://www.reddit.com/r/okanaganbiking/,okanaganbiking_0,Just be a decent person
2,earliest,reddit,1,unchanged,unchanged,2021-04-23 00:00:00,20210301,okanaganbiking,https://www.reddit.com/r/okanaganbiking/,okanaganbiking_1,"don't be a dick, if you disagree with someone ..."
3,earliest,reddit,2,unchanged,unchanged,2021-04-23 00:00:00,20210301,okanaganbiking,https://www.reddit.com/r/okanaganbiking/,okanaganbiking_2,"Reddit is largely anonymous, with that said ag..."
4,earliest,reddit,3,unchanged,unchanged,2021-04-23 00:00:00,20210301,okanaganbiking,https://www.reddit.com/r/okanaganbiking/,okanaganbiking_3,Don't make posts or comments about a specific ...


In [5]:
set(data.change_type)

{'added', 'changed', 'deleted', 'unchanged'}

In [3]:
agg_by_rule = pd.pivot_table(data[['ruleID', 'reg_const_labels']], index=['ruleID'], 
                              columns=['reg_const_labels'], aggfunc=len, fill_value=0)
agg_by_rule

reg_const_labels,constitutive,regulatory
ruleID,,
00games_1,0,4
00games_2,0,1
00games_3,0,7
00saesthetics_1,0,1
00sbabies_0,0,2
...,...,...
zyrocrimson_4,0,2
zyrocrimson_5,0,1
zywieczdrojhomies_0,0,1


In [4]:
change_type = pd.pivot_table(data[['ruleID', 'change_type']], index=['ruleID'], 
                              columns=['change_type'], aggfunc=len, fill_value=0)
agg_by_rule = agg_by_rule.join(change_type, on="ruleID")
agg_by_rule

,constitutive,regulatory,added,change_added,changed,deleted,unchanged
ruleID,,,,,,,
00games_1,0,4,0,0,0,0,4
00games_2,0,1,0,0,0,0,1
00games_3,0,7,0,0,0,0,7
00saesthetics_1,0,1,0,0,0,0,1
00sbabies_0,0,2,0,0,0,0,2
...,...,...,...,...,...,...,...
zyrocrimson_4,0,2,2,0,0,0,0
zyrocrimson_5,0,1,1,0,0,0,0
zywieczdrojhomies_0,0,1,0,0,0,0,1


In [5]:
agg_by_rule_unchanged = agg_by_rule[agg_by_rule.unchanged > 0][['constitutive', 'regulatory']]
agg_by_rule_unchanged['change_type'] = 'unchanged'
agg_by_rule_change_added = agg_by_rule[agg_by_rule.change_added > 0][['constitutive', 'regulatory']]
agg_by_rule_change_added['change_type'] = 'change_added'
agg_by_rule_added = agg_by_rule[agg_by_rule.added > 0][['constitutive', 'regulatory']]
agg_by_rule_added['change_type'] = 'added'
agg_by_rule_changed = agg_by_rule[agg_by_rule.changed > 0][['constitutive', 'regulatory']]
agg_by_rule_changed['change_type'] = 'changed'
agg_by_rule_deleted = agg_by_rule[agg_by_rule.deleted > 0][['constitutive', 'regulatory']]
agg_by_rule_deleted['change_type'] = 'deleted'

In [6]:
reg_const_df = pd.concat([agg_by_rule_unchanged, agg_by_rule_change_added, 
                          agg_by_rule_added, agg_by_rule_deleted, agg_by_rule_changed])
reg_const_df.reset_index(inplace=True)
reg_const_df = reg_const_df.sample(frac=1).reset_index(drop=True)
reg_const_df

,ruleID,constitutive,regulatory,change_type
0,ttsmyf_0,0,3,unchanged
1,homemadedildos_5,0,1,unchanged
2,uwu_gang_official_2,0,1,unchanged
3,mymusicreviews_2,0,9,unchanged
4,lolparler_0,0,2,unchanged
...,...,...,...,...
285854,monsterlove_5,0,3,unchanged
285855,turkspace_8,0,1,added
285856,initiatingspin_1,0,2,unchanged
285857,algorand_3,0,2,unchanged


In [7]:
reg_const_stats = reg_const_df[['change_type', 'constitutive', 'regulatory']].groupby(by=["change_type"]).mean()
reg_const_stats["reg:const"] = reg_const_stats.regulatory/reg_const_stats.constitutive
reg_const_stats

,constitutive,regulatory,reg:const
change_type,,,
added,0.277974,2.343310,8.429947
change_added,0.753488,6.051163,8.030864
changed,0.651855,6.087952,9.339424
deleted,0.263047,2.465775,9.373878
unchanged,0.229932,2.040411,8.873993


In [8]:
reg_const_meta_stats = reg_const_df[['change_type', 'constitutive', 'regulatory']].groupby(by=["change_type"]).sum()
reg_const_meta_stats["% const"] = reg_const_meta_stats.constitutive/(reg_const_meta_stats.regulatory + reg_const_meta_stats.constitutive)
reg_const_meta_stats["% reg"] = reg_const_meta_stats.regulatory/(reg_const_meta_stats.regulatory + reg_const_meta_stats.constitutive)
reg_const_meta_stats

,constitutive,regulatory,% const,% reg
change_type,,,,
added,6752,56919,0.106045,0.893955
change_added,486,3903,0.110731,0.889269
changed,1423,13290,0.096717,0.903283
deleted,1003,9402,0.096396,0.903604
unchanged,58616,520158,0.101276,0.898724
